In [1]:
seed_value = 1986

# Data Preparation

## get audio files

In [2]:
# Your code here
from speechbrain.utils.data_utils import get_all_files

# Your code here
data_files = get_all_files('/home/ulaval.ca/maelr5/scratch/parkinsons', match_and=['.wav'])

print('data size= ', len(data_files))



INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


data size=  831


In [3]:
type(data_files)

list

## split data into train/ valid/ test sets:

80% Training, 10%Validation, 10% Test


In [4]:
from sklearn.model_selection import train_test_split

train_files, test_files = train_test_split(data_files, test_size=0.2, train_size=0.8, random_state=seed_value, shuffle=True)
valid_files, test_files = train_test_split(test_files, test_size=0.5, train_size=0.5, random_state=seed_value, shuffle=True)

print('train size= ', len(train_files))
print('valid size= ', len(valid_files))
print('test size= ', len(test_files))


train size=  664
valid size=  83
test size=  84


In [18]:
import torchaudio
print(torchaudio.info("/home/ulaval.ca/maelr5/scratch/parkinsons/15 Young Healthy Control/Davide S/B1LBULCAAS94M100120171036.wav"))
print(torchaudio.info("/home/ulaval.ca/maelr5/scratch/parkinsons/22 Elderly Healthy Control/GILDA C/D2GCIALSDA52F170320171126.wav"))


AudioMetaData(sample_rate=16000, num_frames=660793, num_channels=1, bits_per_sample=16, encoding=PCM_S)
AudioMetaData(sample_rate=16000, num_frames=110000, num_channels=1, bits_per_sample=16, encoding=PCM_S)


In [17]:
print(torchaudio.info("/home/ulaval.ca/maelr5/scratch/parkinsons/28 People with Parkinson's disease/1-5/Domenico C/B1cdaopmoe67M2605161902.wav"))
print(torchaudio.info("/home/ulaval.ca/maelr5/scratch/parkinsons/28 People with Parkinson's disease/17-28/Giovanni M/FB1GMIAOSVI44M100220170952.wav"))
print(torchaudio.info("/home/ulaval.ca/maelr5/scratch/parkinsons/28 People with Parkinson's disease/6-10/Saverio S/FB1ssacvhei61M1606161751.wav"))



AudioMetaData(sample_rate=44100, num_frames=3244628, num_channels=1, bits_per_sample=16, encoding=PCM_S)
AudioMetaData(sample_rate=16000, num_frames=531500, num_channels=1, bits_per_sample=16, encoding=PCM_S)
AudioMetaData(sample_rate=44100, num_frames=1461474, num_channels=1, bits_per_sample=16, encoding=PCM_S)


## create json files

In [20]:
import json
import torchaudio

def create_json(json_file, audiolist):
  json_dict = {}
  for audiofile in audiolist:

    # Getting info
    audioinfo = torchaudio.info(audiofile)

    # Compute the duration in seconds.
    # This is the number of samples divided by the sampling frequency
    duration = audioinfo.num_frames / audioinfo.sample_rate

    # Get spk Label by manipulating the audio path
    # detection_id = audiofile.split('/')[-4].split(' ')[-2:]
    if "Healthy Control" in audiofile:
        detection_id = "Healthy Control"
    elif "with Parkinson's disease" in audiofile:
        detection_id = "with Parkinson's disease"

    # Get a unique utterance id
    uttid = audiofile.split('/')[-2] + '_' + audiofile.split('/')[-1][:-4]

    # Create entry for this utterance
    json_dict[uttid] = {
            "path": audiofile,
            "length": duration,
            "detection": detection_id,
    }

    # Writing the dictionary to the json file
    with open(json_file, mode="w") as json_f:
      json.dump(json_dict, json_f, indent=2)



create_json('train.json', train_files)
create_json('valid.json', valid_files)
create_json('test.json', test_files)



The json files are formatted in the following way:-

train.json:
```
{
  "Vito S_D1VSIPTIOZ46M240120171923": {
    "path": "/home/ulaval.ca/maelr5/scratch/parkinsons/28 People with Parkinson's disease/11-16/Vito S/D1VSIPTIOZ46M240120171923.wavv",
    "length": 6.65625,
    "detection": "with Parkinson's disease"
  },
  "VITO L_B2VLIATFOO55M300320171238": {
    "path": "/home/ulaval.ca/maelr5/scratch/parkinsons/22 Elderly Healthy Control/VITO L/B2VLIATFOO55M300320171238.wav",
    "length": 71.1339375,
    "detection": "Healthy Control"
  },
  "Giovanni M_B1GMIAOSVI44M100220170942": {
    "path": "/home/ulaval.ca/maelr5/scratch/parkinsons/28 People with Parkinson's disease/17-28/Giovanni M/B1GMIAOSVI44M100220170942.wav",
    "length": 70.912,
    "detection": "with Parkinson's disease"
  },
....
```

valid.json:
```
{
  "GRAZIA G_B2GGRIAUZL45F020420171820": {
    "path": "/home/ulaval.ca/maelr5/scratch/parkinsons/22 Elderly Healthy Control/GRAZIA G/B2GGRIAUZL45F020420171820.wav",
    "length": 62.763125,
    "detection": "Healthy Control"
  },
  "GRAZIA G_VU2GGRIAUZL45F020420171824": {
    "path": "/home/ulaval.ca/maelr5/scratch/parkinsons/22 Elderly Healthy Control/GRAZIA G/VU2GGRIAUZL45F020420171824.wav",
    "length": 5.862625,
    "detection": "Healthy Control"
  },
  "Giulia P_B2GPIUUGLL63F100220171021": {
    "path": "/home/ulaval.ca/maelr5/scratch/parkinsons/28 People with Parkinson's disease/17-28/Giulia P/B2GPIUUGLL63F100220171021.wav",
    "length": 54.1875,
    "detection": "with Parkinson's disease"
  },
....
```

test.json:
```
{
  "ANGELA C_VU2ACNAGRER49F210320170923": {
    "path": "/home/ulaval.ca/maelr5/scratch/parkinsons/22 Elderly Healthy Control/ANGELA C/VU2ACNAGRER49F210320170923.wav",
    "length": 6.547625,
    "detection": "Healthy Control"
  },
  "Vito S_PR1VSIOTLOP47M100220171337": {
    "path": "/home/ulaval.ca/maelr5/scratch/parkinsons/28 People with Parkinson's disease/17-28/Vito S/PR1VSIOTLOP47M100220171337.wav",
    "length": 65.25,
    "detection": "with Parkinson's disease"
  },
  "Nicola S_VE2sncihcio44M1606161722": {
    "path": "/home/ulaval.ca/maelr5/scratch/parkinsons/28 People with Parkinson's disease/6-10/Nicola S/VE2sncihcio44M1606161722.wav",
    "length": 6.5,
    "detection": "with Parkinson's disease"
  },
....
```

## Create small data for sanity check of the model:

The model should overfit this data after training for multiple epochs.

In [5]:
import json
import torchaudio

def create_json(json_file, audiolist):
  json_dict = {}
  for audiofile in audiolist[:2]:

    # Getting info
    audioinfo = torchaudio.info(audiofile)

    # Compute the duration in seconds.
    # This is the number of samples divided by the sampling frequency
    duration = audioinfo.num_frames / audioinfo.sample_rate

    # Get spk Label by manipulating the audio path
    if "Healthy Control" in audiofile:
        detection_id = "Healthy Control"
    elif "with Parkinson's disease" in audiofile:
        detection_id = "with Parkinson's disease"

    # Get a unique utterance id
    uttid = audiofile.split('/')[-2] + '_' + audiofile.split('/')[-1][:-4]

    # Create entry for this utterance
    json_dict[uttid] = {
            "path": audiofile,
            "length": duration,
            "detection": detection_id,
    }

    # Writing the dictionary to the json file
    with open(json_file, mode="w") as json_f:
      json.dump(json_dict, json_f, indent=2)



create_json('train-check.json', train_files)
create_json('valid-check.json', valid_files)
create_json('test-check.json', test_files)



## Data statistics

### number of examples in each class 

to check the classes balance ratio

In [4]:
from speechbrain.utils.data_utils import get_all_files

young_healthy_files = get_all_files("/home/ulaval.ca/maelr5/scratch/parkinsons/15 Young Healthy Control", match_and=['.wav'])
elderly_healthy_files = get_all_files("/home/ulaval.ca/maelr5/scratch/parkinsons/22 Elderly Healthy Control", match_and=['.wav'])

print('15 young healthy data size= ', len(young_healthy_files))
print('22 elderly healthy data size= ', len(elderly_healthy_files))
print('37 candidates - Healthy data size= ', len(young_healthy_files) + len(elderly_healthy_files))


15 young healthy data size=  45
22 elderly healthy data size=  349
37 candidates - Healthy data size=  394


In [5]:
data_files = get_all_files("/home/ulaval.ca/maelr5/scratch/parkinsons/28 People with Parkinson's disease", match_and=['.wav'])

print("28 candidates - with Parkinson's disease data size= ", len(data_files))


28 candidates - with Parkinson's disease data size=  437


In [ ]:

# 39 Healthy (1) in test
# 45 PD (0) in test

# decrease the data and use the short recordings of vowels

In [2]:
# Your code here
from speechbrain.utils.data_utils import get_all_files

# Your code here
data_files = get_all_files('/home/ulaval.ca/maelr5/scratch/parkinsons',
                           match_and=['.wav'],
                           match_or=['VA', 'VE', 'VI', 'VO', 'VU'],
                          )

print('data size= ', len(data_files))

from sklearn.model_selection import train_test_split

train_files, test_files = train_test_split(data_files, test_size=0.2, train_size=0.8, random_state=seed_value, shuffle=True)
valid_files, test_files = train_test_split(test_files, test_size=0.5, train_size=0.5, random_state=seed_value, shuffle=True)

print('train size= ', len(train_files))
print('valid size= ', len(valid_files))
print('test size= ', len(test_files))


data size=  534
train size=  427
valid size=  53
test size=  54


In [3]:
import json
import torchaudio

def create_json(json_file, audiolist):
  json_dict = {}
  for audiofile in audiolist:

    # Getting info
    audioinfo = torchaudio.info(audiofile)

    # Compute the duration in seconds.
    # This is the number of samples divided by the sampling frequency
    duration = audioinfo.num_frames / audioinfo.sample_rate

    # Get spk Label by manipulating the audio path
    if "Healthy Control" in audiofile:
        detection_id = "Healthy Control"
    elif "with Parkinson's disease" in audiofile:
        detection_id = "with Parkinson's disease"

    # Get a unique utterance id
    uttid = audiofile.split('/')[-2] + '_' + audiofile.split('/')[-1][:-4]

    # Create entry for this utterance
    json_dict[uttid] = {
            "path": audiofile,
            "length": duration,
            "detection": detection_id,
    }

    # Writing the dictionary to the json file
    with open(json_file, mode="w") as json_f:
      json.dump(json_dict, json_f, indent=2)



create_json('train-vowels.json', train_files)
create_json('valid-vowels.json', valid_files)
create_json('test-vowels.json', test_files)

